In [1]:
# Use remove command to be able to keep creating the database as the code is written
!rm hawaii.sqlite 

In [2]:
# Import all dependencies and tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import seaborn as sns
from sklearn import preprocessing #Used for Normalized calculations
import pymysql
pymysql.install_as_MySQLdb()
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Database Creation

In [3]:
# Create engine for SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [4]:
# Create a connection to the engine
conn = engine.connect()

In [5]:
# Use `declarative_base` from SQLAlchemy to model the Hawaii Stations table as an ORM class (Station)

Base = declarative_base()

class Station(Base):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column (Float)
    longitude = Column (Float)
    elevation = Column (Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [6]:
class Measurement(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column (Float)
    tobs = Column (Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [7]:
# Create the tables in the database
Base.metadata.create_all(engine)

In [8]:
# Load clean data into dataframes
stations_df = pd.read_csv("Resources/clean_hawaii_stations.csv", low_memory = False)

measurements_df = pd.read_csv("Resources/clean_hawaii_measurements.csv", low_memory = False)

In [9]:
# Show dataframes and types to ensure data reads correctly
stations_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [10]:
stations_df.dtypes

station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object

In [12]:
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [13]:
measurements_df.dtypes 

station     object
date        object
prcp       float64
tobs         int64
dtype: object

In [14]:
# Use Orient='records' to create a list of data to write
stations_data = stations_df.to_dict(orient='records')
measurements_data = measurements_df.to_dict(orient='records')

In [19]:
# Reflect tables using metadata
metadata = MetaData(bind=engine)
metadata.reflect()

In [20]:
# Save new tables
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [23]:
# Insert data into the tables
conn.execute(stations_table.insert(), stations_data)
conn.execute(measurements_table.insert(), measurements_data)

In [24]:
# Make sure data loaded into each table
conn.execute("select * from stations").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [25]:
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]